# #7DaysOfCode - Data Science

# Projeto 1: Análise dos gastos parlamentares (CEAPS) - 2018 a 2022  

Este notebook faz parte do desafio [7 Days of Code](https://7daysofcode.io/), com foco em explorar e transformar dados utilizando a biblioteca **Pandas**.  

O conjunto de dados utilizado contém informações sobre a **Cota para Exercício da Atividade Parlamentar (CEAPS)** no período de **2018 a 2022**, disponibilizados pelo Senado Federal:  
🔗 **Fonte oficial:** [Dados Abertos CEAPS](https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps)  

### Objetivos
O objetivo deste estudo é compreender como os parlamentares utilizam a cota parlamentar ao longo dos anos, buscando responder às seguintes questões:  
- Quais são as principais categorias de gastos?
- Existe um padrão sazonal nos gastos ao longo dos anos?
- Há parlamentares que gastam significativamente mais do que a média?

O CEAPS é um dos mecanismos de transparência do Legislativo, permitindo que qualquer cidadão acompanhe como os recursos públicos estão sendo utilizados. Essa análise pode revelar padrões interessantes e fornecer insights sobre a distribuição desses gastos.  

### Sumário
Notebook 1: Limpeza e preparação dos dados  
- Importação e concatenação dos arquivos de despesas (2018-2022)  
- Remoção de duplicatas e tratamento de valores nulos  
- Ajuste de tipos de dados e normalização de campos (CNPJ/CPF, fornecedores, meses)  
- Identificação e remoção de outliers  
- Exportação dos dados limpos para análise  

Notebook 2: Análise exploratória e storytelling  
- Estatísticas descritivas e identificação de padrões de gastos  
- Comparação entre categorias de despesas e parlamentares  
- Investigação de tendências e sazonalidade nos gastos  
- Criação de visualizações e dashboards para ilustrar insights  
- Construção de uma narrativa baseada nos dados  

Notebook 3: Forecasting (previsão de gastos)  
- Análise temporal dos gastos ao longo dos anos  
- Modelagem estatística para projeção de despesas futuras  
- Uso de algoritmos como ARIMA, Prophet ou modelos de regressão  
- Avaliação da precisão dos modelos e interpretação dos resultados  

# Dia 1: Limpeza e preparação dos dados

# Importações

## 0.1 Bibliotecas e módulos

In [2]:
# Importando as bibliotecas e módulos necessários para análise
import pandas as pd
import glob
import os
import re
from unidecode import unidecode

## 0.2 Dados


#### O que foi feito:
O código concatena arquivos CSV de 2018 a 2022 sobre despesas parlamentares e os salva em uma pasta de dados processados.
1. **Definição de caminhos**: Especifica as pastas de dados brutos e processados.
    -  Parâmetros no `read_csv`:
        - **`encoding='latin1'`**: Evita problemas com caracteres especiais.
        - **`sep=';'`**: Usa ponto e vírgula como separador de colunas (padrão no Brasil).
        - **`quotechar='"'`**: Define aspas duplas como delimitador de valores textuais.
        - **`skiprows=1`**: Ignora a primeira linha, que pode ter informações extras.
        - **`decimal=','`**: Usa vírgula como separador decimal.
2. **Leitura dos arquivos CSV**: Usa `glob` para pegar todos os arquivos `despesa_ceaps_*.csv`.
3. **Concatenação**: Combina os dados de todos os arquivos em um único DataFrame.
4. **Salvamento**: Cria a pasta `processed` (se não existir) e salva o arquivo concatenado como `despesa_ceaps_2018_2022.csv`.

In [3]:
raw_path = "/Users/liviagrigolon/Documents/GitHub/7-days-of-data-science/data/raw/ceaps"
processed_path = "data/processed"

# Encontrar os arquivos CSV na pasta raw
arquivos = glob.glob(os.path.join(raw_path, 'despesa_ceaps_*.csv'))

# Carregar e concatenar os arquivos
df_ceaps = pd.concat([pd.read_csv(arquivo, encoding='latin1', sep=';', quotechar='"', skiprows=1, decimal=',') for arquivo in arquivos], ignore_index=True)

# Criar a pasta processed se não existir
os.makedirs(processed_path, exist_ok=True)

# Salvar o DataFrame final
df_ceaps.to_csv(os.path.join(processed_path, "despesa_ceaps_2018_2022.csv"), index=False)

print("Arquivos compilados e salvos com sucesso!")

Arquivos compilados e salvos com sucesso!


# 1 Limpeza dos dados

## 1.1 Remoção de duplicatas e colunas vazias

In [4]:
# Removendo valores ausentes para garantir qualidade nos dados
df_ceaps = df_ceaps.dropna(axis=1, how='all').drop_duplicates()
print(f"✅ Removidas duplicatas. Linhas restantes: {df_ceaps.shape[0]}\n")

✅ Removidas duplicatas. Linhas restantes: 93848



## 1.2 Tratamento de dados nulos

In [5]:
print("Valores nulos por coluna:\n", df_ceaps.isnull().sum().to_string()) #Conta os nulos por coluna

Valores nulos por coluna:
 ANO                      0
MES                      0
SENADOR                  0
TIPO_DESPESA             0
CNPJ_CPF                 0
FORNECEDOR               0
DOCUMENTO             3979
DATA                     0
DETALHAMENTO         36383
VALOR_REEMBOLSADO        0
COD_DOCUMENTO            0


In [6]:
#Preenchendo valores nulos
df_ceaps['DOCUMENTO'] = df_ceaps['DOCUMENTO'].fillna('Desconhecido')
df_ceaps['DETALHAMENTO'] = df_ceaps['DETALHAMENTO'].fillna('Não informado')
print("✅ Valores nulos preenchidos!\n")

✅ Valores nulos preenchidos!



## 1.3 Revisão dos tipos de dados


#### O que foi feito:
- **`VALOR_REEMBOLSADO` → `float`**  
  - Utilizamos `pd.to_numeric()` para garantir que os valores sejam numéricos, convertendo qualquer erro em `NaN` (`errors='coerce'`).  
  - Isso evita problemas com valores mal formatados no dataset.  

- **`DATA` → `datetime64`**  
  - Utilizamos `pd.to_datetime()` para transformar a coluna de data no formato correto.  
  - O parâmetro `dayfirst=True` garante que o formato seja **DD/MM/YYYY** (padrão brasileiro).  
  - Caso alguma data esteja inválida, ela será convertida para `NaT` (nulo).  

- **Conversão de colunas para `string`**  
  - Definimos explicitamente que algumas colunas são texto (`string`) para evitar problemas de interpretação.  
  - Isso garante que colunas como `SENADOR`, `TIPO_DESPESA`, `CNPJ_CPF`, `FORNECEDOR`, `DOCUMENTO` e `DETALHAMENTO` sejam tratadas corretamente como texto.  

In [7]:
print(df_ceaps.dtypes)  # Checar os tipos de dados

ANO                    int64
MES                    int64
SENADOR               object
TIPO_DESPESA          object
CNPJ_CPF              object
FORNECEDOR            object
DOCUMENTO             object
DATA                  object
DETALHAMENTO          object
VALOR_REEMBOLSADO    float64
COD_DOCUMENTO          int64
dtype: object


In [8]:
# Convertendo tipos de dados para facilitar a análise
df_ceaps['VALOR_REEMBOLSADO'] = pd.to_numeric(df_ceaps['VALOR_REEMBOLSADO'], errors='coerce')  # Garante que os valores de reembolso sejam float
df_ceaps['DATA'] = pd.to_datetime(df_ceaps['DATA'], errors='coerce', dayfirst=True)  # Converte datas
df_ceaps = df_ceaps.astype({
    'SENADOR': 'string', 'TIPO_DESPESA': 'string', 'CNPJ_CPF': 'string',
    'FORNECEDOR': 'string', 'DOCUMENTO': 'string', 'DETALHAMENTO': 'string', 'MES': 'string'
}) # Padroniza colunas como string
print("✅ Tipos de dados ajustados.\n")
display(df_ceaps.dtypes)  # Checar os tipos de dados após as alterações

✅ Tipos de dados ajustados.



ANO                           int64
MES                  string[python]
SENADOR              string[python]
TIPO_DESPESA         string[python]
CNPJ_CPF             string[python]
FORNECEDOR           string[python]
DOCUMENTO            string[python]
DATA                 datetime64[ns]
DETALHAMENTO         string[python]
VALOR_REEMBOLSADO           float64
COD_DOCUMENTO                 int64
dtype: object

## 1.4 Normalização de CNPJ/CPF

#### O que foi feito:
1. Removemos caracteres especiais (pontos, traços, barras) para manter apenas os números.   
2. Contamos a distribuição dos tamanhos dos valores após a normalização.  
3. Identificamos possíveis erros filtrando registros com tamanhos inválidos (diferentes de 11 para CPF ou 14 para CNPJ).  
4. Exibimos uma mensagem de alerta caso valores inválidos sejam encontrados.

In [9]:
# Limpeza diretamente na coluna existente
df_ceaps['CNPJ_CPF'] = df_ceaps['CNPJ_CPF'].str.replace(r'\D', '', regex=True)

# Contagem de caracteres após a limpeza
tamanhos = df_ceaps['CNPJ_CPF'].str.len().value_counts()
print("Distribuição dos tamanhos de CNPJ/CPF:\n", tamanhos, "\n")

# Identificar valores inválidos
df_erro = df_ceaps[~df_ceaps['CNPJ_CPF'].str.len().isin([11, 14])]
if not df_erro.empty:
    print("⚠️ Valores inválidos detectados em CNPJ/CPF:")
    print(df_erro[['CNPJ_CPF']].head())
else:
    print("✅ Todos os CNPJs/CPFs possuem tamanhos corretos (11 ou 14 caracteres).\n")

Distribuição dos tamanhos de CNPJ/CPF:
 CNPJ_CPF
14    91739
11     2109
Name: count, dtype: Int64 

✅ Todos os CNPJs/CPFs possuem tamanhos corretos (11 ou 14 caracteres).



## 1.5 Conversão de meses numéricos para nomes de meses

#### O que foi feito:
Foi utilizada a função `map` do pandas para aplicar essa transformação na coluna `MES` do DataFrame, melhorando a legibilidade dos dados e tornando as análises mais intuitivas.
1. Criação de um dicionário que associa números (1 a 12) aos nomes dos meses.
2. Substituição dos valores na coluna `MES` pelo nome correspondente utilizando o método `map`.
3. Verificação do resultado para garantir que a transformação foi aplicada corretamente.

In [10]:
# Dicionário para mapear números de meses para nomes de meses
meses_map = {
    '1': 'Janeiro', '2': 'Fevereiro', '3': 'Março', '4': 'Abril', '5': 'Maio', '6': 'Junho',
    '7': 'Julho', '8': 'Agosto', '9': 'Setembro', '10': 'Outubro', '11': 'Novembro', '12': 'Dezembro'
}

# Substituindo os números de mês pelos nomes dos meses
df_ceaps['MES'] = df_ceaps['MES'].map(meses_map).astype('string')

# Verificando a mudança
print(df_ceaps[['ANO', 'MES']].head())


    ANO        MES
0  2018    Janeiro
1  2018    Janeiro
2  2018    Janeiro
3  2018    Janeiro
4  2018  Fevereiro


## 1.6 Padronizar nomes dos fornecedores

In [11]:
# Ajustar tipo da coluna para evitar problemas com pandas string dtype
df_ceaps["FORNECEDOR"] = df_ceaps["FORNECEDOR"].astype(str)

# Função para normalizar nomes de fornecedores
def normalizar_fornecedor(nome):
    nome = nome.strip().lower()  # Remover espaços extras e converter para minúsculas
    nome = unidecode(nome)  # Remover acentos
    nome = re.sub(r"[^\w\s]", "", nome)  # Remove pontuação (pontos, vírgulas, etc.)
    nome = re.sub(r"\b(ltda|eireli|me|sa|s/a|s.a)\b", "", nome, flags=re.IGNORECASE)  # Remove tipos de empresa
    nome = re.sub(r"\s+", " ", nome)  # Substituir múltiplos espaços por um único
    return nome.strip()  # Retorna o nome limpo

# Aplicar a normalização
df_ceaps["FORNECEDOR"] = df_ceaps["FORNECEDOR"].apply(normalizar_fornecedor)

# Verificar os valores normalizados
print(df_ceaps["FORNECEDOR"].value_counts().head(10))  # Mostra os 10 mais frequentes

FORNECEDOR
adria viagens e turismo                                                                         6515
latam                                                                                           2554
gol                                                                                             1329
telefonica brasil                                                                                894
claro                                                                                            866
uber do brasil tecnologia                                                                        835
lm turismo                                                                                       645
sindicato dos permissionarios de taxis e motoristas auxiliares do distrito federal sinpetaxi     617
azul                                                                                             594
posto de gasolina e garagem echeverria                                          

## 1.7 Correção de valores monetários

### 1.7.1 Análise estatística básica

In [12]:
descricao = df_ceaps['VALOR_REEMBOLSADO'].describe()
print("Estatísticas básicas de VALOR_REEMBOLSADO:\n", descricao, "\n")
#display(df_ceaps[df_ceaps['VALOR_REEMBOLSADO'] < 0])  # Confere se há valores negativos

Estatísticas básicas de VALOR_REEMBOLSADO:
 count     93848.000000
mean       1317.147702
std        3007.744489
min           0.010000
25%         150.000000
50%         392.870000
75%        1469.155000
max      120000.000000
Name: VALOR_REEMBOLSADO, dtype: float64 



### 1.7.2 Identificar outliers

In [13]:
df_ceaps.groupby('TIPO_DESPESA')['VALOR_REEMBOLSADO'].describe()  # Outliers entre tipo de despesa e valor reembolsado

,count,mean,std,min,25%,50%,75%,max
TIPO_DESPESA,,,,,,,,
"Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.",20248.0,1046.081178,1560.366122,0.02,172.91,381.735,1095.43,15000.00
"Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos.",6768.0,690.750197,1379.383090,0.01,66.30,199.900,570.00,40000.00
"Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar",5056.0,5804.374840,8566.408199,3.56,800.00,2500.000,7500.00,120000.00
Divulgação da atividade parlamentar,5237.0,2957.428726,5154.072238,27.00,1000.00,1500.000,2950.00,103900.00
"Locomoção, hospedagem, alimentação, combustíveis e lubrificantes",35665.0,656.183278,1840.321478,0.03,99.90,173.290,286.40,66730.00
"Passagens aéreas, aquáticas e terrestres nacionais",20717.0,1409.834754,919.923201,0.01,735.53,1270.500,1851.89,10871.78
Serviços de Segurança Privada,157.0,1976.635860,4075.691955,6.57,231.45,440.240,1800.00,23986.02


### 1.7.3 Remoção de outliers com Intervalo Interquartil (IQR)

In [14]:
# Definir limites para outliers
Q1 = df_ceaps['VALOR_REEMBOLSADO'].quantile(0.25)  # Primeiro quartil (25%)
Q3 = df_ceaps['VALOR_REEMBOLSADO'].quantile(0.75)  # Terceiro quartil (75%)
IQR = Q3 - Q1  # Intervalo interquartil

# Definir os limites inferior e superior
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar os dados, removendo outliers
df_ceaps_limpo = df_ceaps[
    (df_ceaps['VALOR_REEMBOLSADO'] >= limite_inferior) & 
    (df_ceaps['VALOR_REEMBOLSADO'] <= limite_superior)
]

# Ver quantas linhas sobraram
print(f"Removendo outliers...\nLinhas antes: {len(df_ceaps)}\nLinhas depois: {len(df_ceaps_limpo)}\nRemovidos: {len(df_ceaps) - len(df_ceaps_limpo)} outliers.\n")
print("✅ Dados limpos e prontos para análise!\n")

Removendo outliers...
Linhas antes: 93848
Linhas depois: 85857
Removidos: 7991 outliers.

✅ Dados limpos e prontos para análise!



# 2 Exportar os dados limpos

In [15]:
# Salvar o DataFrame limpo em um arquivo pickle
df_ceaps_limpo.to_pickle('data/processed/despesa_ceaps_2018_2022_limpo.pkl')
print("Dados salvos na pasta 'data/processed'")

Dados salvos na pasta 'data/processed'


## Conclusão

- No primeiro dia do desafio, realizamos a limpeza e preparação dos dados, removendo valores ausentes e ajustando os tipos de dados para garantir qualidade na análise.
- Com os dados tratados, agora podemos partir para a **análise exploratória**, onde começaremos a investigar padrões e identificar insights interessantes.
- A ideia é gerar visualizações para criar uma narrativa sobre os dados, respondendo perguntas e apresentando os resultados de maneira clara e impactante.

🔜 **Próximos passos:**  
A próxima etapa será a **análise exploratória** seguida da criação de gráficos e visualizações que nos ajudem a contar uma história a partir dos dados. Vamos focar em descobrir padrões e insights relevantes.